# Assignment: SVD Preprocessing on MNIST with Logistic Regression

## Instructions:
In this assignment, you will apply **Singular Value Decomposition (SVD)** as a preprocessing step to the **MNIST dataset** and train a **logistic regression classifier**. You will compare the model performance and training time when using different levels of SVD for dimensionality reduction.

In this assignment, you will need to:
1. Load the MNIST dataset and normalize it.
2. Perform SVD and reduce the dimensions of the data.
3. Train a logistic regression model on the original and SVD-reduced data.
4. Measure and compare the training time and accuracy of the model with varying SVD components.
5. Plot the results and analyze how SVD impacts the performance and efficiency of the model.

***
Your tasks include:
1. Implement SVD algorithm. You are not allowed to directly use SVD implemented by other packages, but you may use functions in NumPy. (Part 2)
2. Explore the accuracy and time performance from different numbers of SVD components. (Part 4)
3. Visualize the accuracy, time performance and top 5 singular vectors in the dataset, analyze and explain which number of SVD component looks best to you? (Part 4,5&6) Hint: singular vectors should be reshaped to 28x28 images for visualization.
***
**Note that you may not import any other function or package.** Let's get started!


## Part 1: Load the MNIST dataset and preprocess the data

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn.metrics import accuracy_score, classification_report

# Load MNIST dataset
print("Loading MNIST dataset...")
mnist = fetch_openml('mnist_784', version=1)
X = mnist.data
y = mnist.target

# Normalize the data
X = X / 255.0

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Done!")


Loading MNIST dataset...
Done!


## Part 2: Implement SVD for Dimensionality Reduction

In [10]:
import numpy as np

def svd(X):
    """
    Compute the Singular Value Decomposition (SVD) of matrix X from scratch.
    X = U * S * V^T
    """
    # Step 1: Compute the covariance matrix A^T * A
    A_T_A = np.dot(X.T, X)

    # Step 2: Compute eigenvalues and eigenvectors of A^T * A (for V and S)
    eigenvalues, V = np.linalg.eig(A_T_A)

    # Step 3: Sort the eigenvalues and eigenvectors by decreasing eigenvalue magnitude
    idx = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[idx]
    V = V[:, idx]

    # Step 4: Compute singular values (S is the square root of eigenvalues)
    singular_values = np.sqrt(np.abs(eigenvalues))
    
    # Step 5: Compute U using U = A * V * Σ^-1
    # Filter out eigenvalues that are too small (to avoid division by zero)
    tolerance = 1e-10
    singular_values_inv = np.array([1/s if s > tolerance else 0 for s in singular_values])
    Sigma_inv = np.diag(singular_values_inv)
    
    U = np.dot(X, np.dot(V, Sigma_inv))

    # Step 6: Construct the S matrix as a diagonal matrix of singular values
    S = np.diag(singular_values)

    return U, S, V.T


In [21]:
def apply_svd_custom(X_train, X_test, n_components):
    U, S, VT = svd(X_train)

    U_k = U[:, :n_components]
    S_k = np.diag(S[:n_components])
    VT_k = VT[:n_components, :]

    X_train_reduced = np.dot(U_k, S_k)
    X_test_reduced = np.dot(X_test, VT_k.T)

    return X_train_reduced, X_test_reduced
    

In [22]:
#Checking for correct implementation of SVD
X = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9],
              [10, 11, 12],
              [13, 14, 15]])
U, S, VT = svd(X)
print(U)
print(S)
print(VT)

print(np.matmul(np.matmul(U, S), VT))

[[-1.01345996e-01  7.67938141e-01 -8.12578946e-08]
 [-2.48568753e-01  4.88071281e-01 -5.07570803e-08]
 [-3.95791510e-01  2.08204420e-01 -1.93249434e-08]
 [-5.43014267e-01 -7.16624412e-02  1.39698386e-08]
 [-6.90237024e-01 -3.51529302e-01  3.98140401e-08]]
[[3.51826483e+01 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.47690770e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 2.18272661e-07]]
[[-0.51927261 -0.57552072 -0.63176883]
 [-0.75079244 -0.04592639  0.65893966]
 [ 0.40824829 -0.81649658  0.40824829]]
[[ 1.  2.  3.]
 [ 4.  5.  6.]
 [ 7.  8.  9.]
 [10. 11. 12.]
 [13. 14. 15.]]


## Part 3: Train Logistic Regression and Measure Performance

In [23]:
# Function to train logistic regression and track training time
def train_logistic_regression(X_train, y_train, X_test, y_test):
    model = LogisticRegression(max_iter=1000, solver='saga', random_state=42, multi_class='multinomial')
    
    # Measure training time
    start_time = time.time()
    model.fit(X_train, y_train)
    training_time = time.time() - start_time
    
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    return accuracy, training_time


## Part 4: Experiment with Different Levels of SVD

Now, apply SVD with varying numbers of components and observe how the dimensionality reduction impacts the model's performance. Record both the accuracy and training time for each number of components.


In [24]:
svd_components = [784]  # You need to decide what number to search...

# Store the results
results = []

print("Training models with different levels of SVD preprocessing...")
for n_components in svd_components:
    print(f"Applying custom SVD with {n_components} components...")

    X_train_svd, X_test_svd = apply_svd_custom(X_train, X_test, n_components )
    # Apply SVD to the training and test sets
    # Call apply_svd_custom() here...
    
    # Train the logistic regression model and get accuracy and training time
    accuracy, training_time = train_logistic_regression(X_train_svd, y_train, X_test_svd, y_test)
        
    print(f"SVD components: {n_components}, Accuracy: {accuracy:.4f}, Training time: {training_time:.4f} seconds")


Training models with different levels of SVD preprocessing...
Applying custom SVD with 784 components...


ValueError: Expected 2D array, got 1D array instead:
array=[  0.72365219 -11.01953575 -10.31621056 ...  -1.07456201 -12.88511593
  -3.99237126].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

## Part 5: Visualize and Analyze the Results

Finally, plot the accuracy, training time as a function of the number of SVD components, and top 5 singular vectors. This will help you understand the trade-off between dimensionality reduction, accuracy, and model training time, and how SVD generally works. Hint: singular vectors should be reshaped to 28x28 images for visualization.


In [ ]:
## Your implementation here...
## You may add necessary lines in Part 4 to access data for visualization

## Part 6: Analyze / Conclusion 

YOUR ANSWER: 